In [1]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("YoungWon")

LangSmith 추적을 시작합니다.
[프로젝트명]
YoungWon


In [2]:
from langchain_teddynote.messages import stream_response

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field   # 버전 오류 

openai_api_key=os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    api_key=openai_api_key,
    temperature=0, 
    model_name="gpt-4-turbo"
)

In [12]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [13]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

1. 김철수 상무는 바이크코퍼레이션 소속으로, 이은채 대리에게 이메일을 보냈습니다.
2. 김철수 상무는 이은채 대리의 회사인 테디인터내셔널의 신규 자전거 "ZENESIS"에 관심을 표명하고, 자전거의 기술 사양, 배터리 성능, 디자인에 대한 상세한 브로슈어를 요청했습니다.
3. 김철수 상무는 ZENESIS 모델의 유통 전략과 마케팅 계획을 구체화하기 위해 이 정보가 필요하다고 언급했습니다.
4. 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안했으며, 이은채 대리의 사무실에서 만나기를 원합니다.

In [16]:
from pydantic import BaseModel, Field

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [18]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

# chain 생성
chain = prompt | llm

In [19]:
# chain을 실행하고 결과를 출력한다.
response = chain.stream(
    {
    "email_conversation": email_conversation,
    "question": "이메일 내용중 주요 내용을 추출해 주세요"
    }
)

# 결과는 JSON 형태로 출력됨
output = stream_response(response, return_output=True)

```json
{
  "person": "김철수 상무",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "김철수 상무는 바이크코퍼레이션에서 근무하며, 귀사의 신규 자전거 'ZENESIS'에 대한 상세한 브로슈어를 요청하고, 기술 사양, 배터리 성능, 디자인 정보를 필요로 합니다. 이를 통해 유통 전략과 마케팅 계획을 구체화하고자 합니다. 또한, 협력 가능성을 논의하기 위해 다음 주 화요일 오전 10시에 미팅을 제안합니다.",
  "date": "1월 15일 오전 10시"
}
```

In [20]:
# PydanticOutputParser를 사용하여 결과를 파싱
structured_output = parser.parse(output)
print(structured_output)

person='김철수 상무' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary="김철수 상무는 바이크코퍼레이션에서 근무하며, 귀사의 신규 자전거 'ZENESIS'에 대한 상세한 브로슈어를 요청하고, 기술 사양, 배터리 성능, 디자인 정보를 필요로 합니다. 이를 통해 유통 전략과 마케팅 계획을 구체화하고자 합니다. 또한, 협력 가능성을 논의하기 위해 다음 주 화요일 오전 10시에 미팅을 제안합니다." date='1월 15일 오전 10시'


In [21]:
# 출력 파서를 추가하여 전체 체인을 재구성
chain = prompt | llm | parser

# chain을 실행하고 결과를 출력
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요",
    }
)
response

EmailSummary(person='김철수 상무', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무는 바이크코퍼레이션에서 근무하며, 귀사의 신규 자전거 'ZENESIS'에 대한 상세한 브로슈어를 요청하고, 기술 사양, 배터리 성능, 디자인 정보를 필요로 합니다. 이를 통해 유통 전략과 마케팅 계획을 구체화하고자 합니다. 또한, 협력 가능성을 논의하기 위해 다음 주 화요일 오전 10시에 미팅을 제안합니다.", date='1월 15일 오전 10시')

### with_structured_output()

In [22]:
llm_with_structered = ChatOpenAI(
    api_key=openai_api_key,
    temperature=0, 
    model_name="gpt-4o"
).with_structured_output(EmailSummary)

In [23]:
# invoke() 함수를 호출하여 결과를 출력
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 ZENESIS 자전거 모델에 대한 브로슈어 요청과 유통 협력 논의를 위한 미팅을 제안합니다. 미팅은 다음 주 화요일(1월 15일) 오전 10시에 제안되었습니다.', date='1월 15일 오전 10시')

### CommaSeparatedListOutputParser

In [28]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}. \n{format_instructions}",
    input_variables=["subject"],    # 입력 변수로 'subjecft' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0, model="gpt-4o")

# 체인 생성
chain = prompt | model | output_parser

In [29]:
chain.invoke({"subject": "대한민국 관광명소"})

['경복궁', '한라산', '부산 해운대', '경주 불국사', '서울 남산타워']

In [30]:
for s in chain.stream({"subject": "대한민국 관광명소"}):
    print(s)

['경복궁']
['한라산']
['부산 해운대']
['경주 불국사']
['제주도 성산일출봉']


### StructuredOutputParser

In [31]:
# ResponseSchema를 사용하여 사용자의 질문에 대한 답변과 사용된 소스에 대한 설명을 포함하는 응답 스키마를 정의
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [32]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [33]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})


{'answer': '서울', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

In [34]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

{'answer': '세종대왕은 한글을 창제하고 문화를 발전시키는 등 다양한 업적을 가지고 있습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}


### JsonOutputParser

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [41]:
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [42]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [43]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행


{'description': '지구 온난화는 지구의 평균 기온이 지속적으로 상승하는 현상으로, 이는 주로 인간 활동에 의해 발생하는 온실가스 배출로 인해 발생합니다. 이로 인해 극지방의 빙하가 녹고, 해수면이 상승하며, 기후 패턴이 변화하고 있습니다. 이러한 변화는 생태계와 인간 사회에 심각한 영향을 미치고 있습니다.',
 'hashtags': '#지구온난화 #기후변화 #온실가스'}

In [44]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)


{'description': '지구 온난화는 대기 중 온실가스 농도의 증가로 인해 지구의 평균 기온이 상승하는 현상을 말합니다. 이는 주로 화석 연료의 연소, 산림 파괴, 산업 활동 등 인간 활동에 의해 발생합니다. 지구 온난화는 기후 변화, 해수면 상승, 생태계 파괴 등 다양한 환경 문제를 초래합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#해수면상승']}


### PandasDataFrameOutputParser

In [45]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [46]:
# ChatOpenAI 모델 초기화 (gpt-3.5-turbo 모델 사용을 권장)
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [47]:
# 출력 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [48]:
df = pd.read_csv("./titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [49]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [58]:
# 열 작업 예시입니다.
df_query = "Age column 을 조회해 주세요."


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 출력
format_parser_output(parser_output)

{'Age': {0: 34.5,
         1: 47.0,
         2: 62.0,
         3: 27.0,
         4: 22.0,
         5: 14.0,
         6: 30.0,
         7: 26.0,
         8: 18.0,
         9: 21.0,
         10: nan,
         11: 46.0,
         12: 23.0,
         13: 63.0,
         14: 47.0,
         15: 24.0,
         16: 35.0,
         17: 21.0,
         18: 27.0,
         19: 45.0,
         20: 55.0,
         21: 9.0,
         22: nan,
         23: 21.0,
         24: 48.0,
         25: 50.0,
         26: 22.0,
         27: 22.5,
         28: 41.0,
         29: nan,
         30: 50.0,
         31: 24.0,
         32: 33.0,
         33: nan,
         34: 30.0,
         35: 18.5,
         36: nan,
         37: 21.0,
         38: 25.0,
         39: nan,
         40: 39.0,
         41: nan,
         42: 41.0,
         43: 30.0,
         44: 45.0,
         45: 25.0,
         46: 45.0,
         47: nan,
         48: 60.0,
         49: 36.0,
         50: 24.0,
         51: 27.0,
         52: 20.0,
         53:

In [51]:
# 행 조회 예시입니다.
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
format_parser_output(parser_output)

{'0': {'Age': 34.5,
       'Cabin': nan,
       'Embarked': 'Q',
       'Fare': 7.8292,
       'Name': 'Kelly, '
               'Mr. '
               'James',
       'Parch': 0,
       'PassengerId': 892,
       'Pclass': 3,
       'Sex': 'male',
       'SibSp': 0,
       'Survived': 0,
       'Ticket': '330911'}}


In [52]:
# row 0 ~ 4의 평균 나이를 구합니다.
df["Age"].head().mean()

38.5

In [53]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)

{'mean': 38.5}


In [54]:
# 잘못 형식화된 쿼리의 예시입니다.
df_query = "Calculate average `Fare` rate."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)

{'mean': 35.627188489208635}


In [55]:
# 결과 검증
df["Fare"].mean()

35.627188489208635

### DatetimeOutputParser

In [59]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },  # 지침을 템플릿에 적용
)

# 프롬프트 내용을 출력
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 0066-04-21, 0448-01-03, 1176-04-06\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

In [60]:
# Chain 을 생성합니다.
chain = prompt | ChatOpenAI() | output_parser

# 체인을 호출하여 질문에 대한 답변을 받습니다.
output = chain.invoke({"question": "Google 이 창업한 연도는?"})


In [61]:
# 결과를 문자열로 변환
output.strftime("%Y-%m-%d")

'1998-09-04'

### EnumOutputParser

In [62]:
from langchain.output_parsers.enum import EnumOutputParser

In [63]:
# enum 모듈을 사용하여 Colors 클래스를 정의
from enum import Enum

# Enum을 상속받으며, RED, GREEN, BLUE 세가지 색상을 가짐
class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

In [64]:
# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Colors)

In [65]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용합니다.
).partial(instructions=parser.get_format_instructions())
# 프롬프트와 ChatOpenAI, 파서를 연결합니다.
chain = prompt | ChatOpenAI() | parser


In [67]:
response = chain.invoke({"object": "하늘"})  # "하늘" 에 대한 체인 호출 실행
print(response)

Colors.BLUE


### OutputFixingParser

In [68]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List


class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(
        description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [69]:
# 잘못된 형식을 일부러 입력
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

# 잘못된 형식으로 입력된 데이터를 파싱하려고 시도
parser.parse(misformatted)

# 오류 출력

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}

In [70]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [71]:
# 잘못된 형식의 출력
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [72]:
actor = new_parser.parse(misformatted)

KeyError: "Input to PromptTemplate is missing variables {'instructions'}.  Expected: ['completion', 'error', 'instructions'] Received: ['completion', 'error']\nNote: if you intended {instructions} to be part of the string and not a variable, please escape it with double curly braces like: '{{instructions}}'."

In [ ]:
actor